In [1]:
import pandas as pd
import numpy as np
from docxtpl import DocxTemplate

In [3]:
df = pd.read_excel(r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\63 suburban schools.xlsx", dtype={'dich':str, 'iin':str})

## Data processing

In [4]:
df.drop(columns=['school_kz', 'test', 'liter', 'note'], inplace=True)
df.rename(columns={'school_ru' : 'school', 'iin' : 'student_id', 'fio' : 'name', 'dich' : 'answers'}, inplace=True)

In [5]:
# Removing white spaces
df = df.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)

In [6]:
lang_dict = {
    1 : 'Казахский',
    2 : 'Русский',
    3 : 'Английский'
}

In [7]:
df['lang'] = df['lang'].map(lang_dict)
df['lang_test'] = df['lang_test'].map(lang_dict)

In [8]:
df['subject'] = df['subject'].apply(lambda x: x + '_4' if '4' in x else x + '_8')
df['subject'] = df['subject'].str[1:]

Conclusion based on elements

In [9]:
elem = pd.read_excel(r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\elements.xlsx")
elem['subject'] = elem['subject'].apply(lambda x: x + '_4' if '4' in x else x + '_8')
elem['subject'] = elem['subject'].str[1:]
elem['element'] = elem['element'].str.strip()
elem['element'] = elem['element'].str.replace('  ', ' ')
elem['element'] = elem['element'].str.replace('\n', '')

In [10]:
answers_cols = np.arange(1,21)
df[answers_cols] = df['answers'].apply(lambda x: pd.Series(list(x)))
df

,school,grade,subject,lang,lang_test,student_id,name,answers,score,1,...,11,12,13,14,15,16,17,18,19,20
0,"КГУ ""IT школа-лицей №7""",8,ENFL_8,Казахский,Английский,100124650803,Нартай Шапағат Мырзаханқызы,00010100010001000100,5,0,...,0,0,0,1,0,0,0,1,0,0
1,"КГУ ""СШ имени Маяковского""",8,ENFL_8,Казахский,Английский,100128652811,Асхатқызы Аяна,00010000000001100001,4,0,...,0,0,0,1,1,0,0,0,0,1
2,"КГУ ""IT школа-лицей №7""",8,ENFL_8,Казахский,Английский,100130650509,Ордабек Аяулым Хасанғалиқызы,10011111111001111010,14,1,...,1,0,0,1,1,1,1,0,1,0
3,"КГУ ""Средняя школа имени Т.Жармагамбетова""",8,RUSL_8,Казахский,Русский,100616553638,Өтеміс Ерасыл Темірбекұлы,11010111110101910011,13,1,...,0,1,0,1,9,1,0,0,1,1
4,"КГУ ""Школа-гимназия №3 города Кандыагаш""",4,SCIE_4,Казахский,Казахский,140210505727,Айсұлтан Әділет,01010111011010000111,11,0,...,1,0,1,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39622,"КГУ ""Школа-гимназия имени Абая"" государственно...",8,RUSL_8,Казахский,Русский,1005276547748,Навирдинова Дильяара Рустамовна,11000111111111111000,14,1,...,1,1,1,1,1,1,1,0,0,0
39623,"КГУ ""Средняя школа имени Х. Досмұхамедова""",8,RUSL_8,Казахский,Русский,1006136511710,Батырғалиқызы Айша,10010101110111111111,15,1,...,0,1,1,1,1,1,1,1,1,1
39624,"КГУ ""Школа-гимназия имени Абая"" государственно...",8,RUSL_8,Казахский,Русский,1007003654754,Бағдатқызы Жанайым,10011101110111110101,14,1,...,0,1,1,1,1,1,0,1,0,1
39625,"КГУ ""Шубаркудыкская общеобразовательная средня...",8,RUSL_8,Казахский,Русский,1100331551286,Cерік Ислам Бауржанұлы,00000100000100090001,3,0,...,0,1,0,0,0,9,0,0,0,1


In [11]:
for col in df[answers_cols]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [12]:
# Counting correct answers
df['correct'] = df.apply(lambda row: df.columns[row == 1].tolist(), axis=1)

In [13]:
schools = df['school'].unique()
subjects = df['subject'].unique()
names = df['name'].unique()

In [33]:
nums = np.arange(1, 21)
school_dict = {}
for school in schools:
    subject_dict = {}
    for subject in subjects:
        subject_df = df[(df['school'] == school) & (df['subject'] == subject)]
        correct_counts = {num: 0 for num in nums}
        for _, row in subject_df.iterrows():
            for digit in row['correct']:
                if digit in nums:
                    correct_counts[digit] += 1
        subject_dict[subject] = correct_counts
    school_dict[school] = subject_dict

In [39]:
school_dict

{'КГУ "IT школа-лицей №7"': {'ENFL_8': {1: 74,
   2: 41,
   3: 27,
   4: 87,
   5: 44,
   6: 79,
   7: 91,
   8: 40,
   9: 28,
   10: 42,
   11: 95,
   12: 71,
   13: 63,
   14: 92,
   15: 50,
   16: 60,
   17: 44,
   18: 33,
   19: 59,
   20: 54},
  'RUSL_8': {1: 43,
   2: 55,
   3: 16,
   4: 107,
   5: 36,
   6: 121,
   7: 62,
   8: 78,
   9: 67,
   10: 119,
   11: 68,
   12: 89,
   13: 78,
   14: 73,
   15: 115,
   16: 98,
   17: 13,
   18: 119,
   19: 88,
   20: 86},
  'SCIE_4': {1: 39,
   2: 50,
   3: 88,
   4: 93,
   5: 25,
   6: 138,
   7: 87,
   8: 89,
   9: 50,
   10: 82,
   11: 102,
   12: 43,
   13: 45,
   14: 73,
   15: 16,
   16: 103,
   17: 28,
   18: 68,
   19: 19,
   20: 102},
  'CHEM_8': {1: 42,
   2: 81,
   3: 36,
   4: 42,
   5: 12,
   6: 31,
   7: 16,
   8: 75,
   9: 42,
   10: 15,
   11: 29,
   12: 13,
   13: 28,
   14: 49,
   15: 65,
   16: 29,
   17: 42,
   18: 23,
   19: 57,
   20: 26},
  'PHYS_8': {1: 37,
   2: 62,
   3: 74,
   4: 36,
   5: 76,
   6: 79,
   7: 

In [50]:
dfs = []
for school, subjects in school_dict.items():
    for subject, counts in subjects.items():
        df = pd.DataFrame(counts, index=[0])  # Assuming counts is a dictionary
        df.index.name = 'School_Subject'
        df.columns.name = 'Number'
        df['School'] = school
        df['Subject'] = subject
        df = df.reset_index().set_index(['School', 'Subject', 'School_Subject'])
        dfs.append(df)

result_df = pd.concat(dfs)
result_df.reset_index(inplace=True)
result_df.drop(columns='School_Subject', inplace=True)

In [51]:
result_df

Number,School,Subject,1,2,3,4,5,6,7,8,...,11,12,13,14,15,16,17,18,19,20
0,"КГУ ""IT школа-лицей №7""",ENFL_8,74,41,27,87,44,79,91,40,...,95,71,63,92,50,60,44,33,59,54
1,"КГУ ""IT школа-лицей №7""",RUSL_8,43,55,16,107,36,121,62,78,...,68,89,78,73,115,98,13,119,88,86
2,"КГУ ""IT школа-лицей №7""",SCIE_4,39,50,88,93,25,138,87,89,...,102,43,45,73,16,103,28,68,19,102
3,"КГУ ""IT школа-лицей №7""",CHEM_8,42,81,36,42,12,31,16,75,...,29,13,28,49,65,29,42,23,57,26
4,"КГУ ""IT школа-лицей №7""",PHYS_8,37,62,74,36,76,79,25,35,...,4,18,34,26,54,8,38,34,40,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
814,КГУ «Общеобразовательная школа № 3 поселка Шор...,KZFL_8,30,31,21,15,21,8,18,27,...,17,23,23,14,12,5,10,12,15,15
815,КГУ «Общеобразовательная школа № 3 поселка Шор...,MATH_4,11,30,10,10,21,22,9,24,...,23,8,23,5,21,18,19,14,16,18
816,КГУ «Общеобразовательная школа № 3 поселка Шор...,KZFL_4,14,6,15,30,16,16,13,16,...,13,29,11,4,15,9,14,16,12,12
817,КГУ «Общеобразовательная школа № 3 поселка Шор...,RUFL_4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Application to report

In [35]:
content = {
    'ENFL_4' : 'Английский язык',
    'ENFL_8' : 'Английский язык',
    'KZFL_4' : 'Казахский язык (Я1)',
    'KZFL_8' : 'Казахский язык (Я1)',
    'KZSL_4' : 'Казахския язык (Я2)',
    'KZSL_8' : 'Казахския язык (Я2)',
    'RUFL_4' : 'Русский язык (Я1)',
    'RUFL_8' : 'Русский язык (Я1)',
    'RUSL_4' : 'Русский язык (Я2)',
    'RUSL_8' : 'Русский язык (Я2)',
    'SCIE_4' : 'Естествознание и познание мира',
    'SCIE_8' : 'Естествознание и познание мира',
    'MATH_4' : 'Математика',
    'MATH_8' : 'Математика',
    'CHEM_4' : 'Химия',
    'CHEM_8' : 'Химия',
    'PHYS_4' : 'Физика',
    'PHYS_8' : 'Физика',
    'BIOL_4' : 'Биология',
    'BIOL_8' : 'Биология'
}

Saving reports

In [34]:
import re

for school in schools:

    sanitized_school_name = re.sub(r'[":№]', '', school)
    output_path = rf"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\Samples\Output\{sanitized_school_name}.docx"

    doc = DocxTemplate(r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\Samples\Report sample.docx")
    
    grouped = df.groupby(['school', 'subject'])['score'].mean().reset_index().round(2)

    for subject in subjects:
        avg = round(grouped[(grouped['school'] == school) & (grouped['subject'] == subject)]['score'] / 20 * 100).tolist() # !Double-checked - CORRECT!
        content[f"{subject}_AVG"] = avg[0] if not len(avg) == 0 else avg

    doc.render(content) 
    doc.save(output_path)

In [36]:
for school in schools:
    grouped = df.groupby(['school', 'subject'])['score'].mean().reset_index().round(2)
    for subject in subjects:
        avg = round(grouped[(grouped['school'] == school) & (grouped['subject'] == subject)]['score'] / 20 * 100).tolist()
        content[f"{subject}_AVG"] = avg[0] if not len(avg) == 0 else avg

In [38]:
content

{'ENFL_4': 'Английский язык',
 'ENFL_8': 'Английский язык',
 'KZFL_4': 'Казахский язык (Я1)',
 'KZFL_8': 'Казахский язык (Я1)',
 'KZSL_4': 'Казахския язык (Я2)',
 'KZSL_8': 'Казахския язык (Я2)',
 'RUFL_4': 'Русский язык (Я1)',
 'RUFL_8': 'Русский язык (Я1)',
 'RUSL_4': 'Русский язык (Я2)',
 'RUSL_8': 'Русский язык (Я2)',
 'SCIE_4': 'Естествознание и познание мира',
 'SCIE_8': 'Естествознание и познание мира',
 'MATH_4': 'Математика',
 'MATH_8': 'Математика',
 'CHEM_4': 'Химия',
 'CHEM_8': 'Химия',
 'PHYS_4': 'Физика',
 'PHYS_8': 'Физика',
 'BIOL_4': 'Биология',
 'BIOL_8': 'Биология',
 'ENFL_8_AVG': 30.0,
 'RUSL_8_AVG': 40.0,
 'SCIE_4_AVG': 32.0,
 'CHEM_8_AVG': 24.0,
 'PHYS_8_AVG': 24.0,
 'MATH_8_AVG': 37.0,
 'KZSL_8_AVG': [],
 'BIOL_8_AVG': 32.0,
 'KZFL_8_AVG': 39.0,
 'MATH_4_AVG': 40.0,
 'KZFL_4_AVG': 32.0,
 'RUFL_4_AVG': [],
 'RUFL_8_AVG': []}

In [39]:
g = df.groupby(['school', 'subject'])['score'].mean().reset_index()
g

,school,subject,score
0,"ГКУ ""Школа-гимназия имени Б.Бейсекбаева с дошк...",BIOL_8,7.709677
1,"ГКУ ""Школа-гимназия имени Б.Бейсекбаева с дошк...",CHEM_8,9.066667
2,"ГКУ ""Школа-гимназия имени Б.Бейсекбаева с дошк...",ENFL_8,9.983607
3,"ГКУ ""Школа-гимназия имени Б.Бейсекбаева с дошк...",KZFL_4,6.192308
4,"ГКУ ""Школа-гимназия имени Б.Бейсекбаева с дошк...",KZFL_8,10.250000
...,...,...,...
674,ОШ имени Ә.Сығай (№13),MATH_4,9.170213
675,ОШ имени Ә.Сығай (№13),MATH_8,8.277778
676,ОШ имени Ә.Сығай (№13),PHYS_8,5.191176
677,ОШ имени Ә.Сығай (№13),RUSL_8,9.246377
